In [55]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np

In [56]:
# 1. Đọc dữ liệu
df = pd.read_csv("encoded_data.csv")

# Tách các feature và target
X = df.drop(columns=['gross','gross_log'])
y = df['gross_log']

# Chuẩn hóa dữ liệu (StandardScaler)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Chia dữ liệu thành train và test để kiểm tra overfitting
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(f"Min gross: {y.min()}")
print(f"Max gross: {y.max()}")
print(f"Mean gross: {y.mean()}")


Min gross: 2.995732273553991
Max gross: 21.79611943720888
Mean gross: 16.81006966826664


In [62]:
#  Khởi tạo mô hình Gradient Boosting
model = GradientBoostingRegressor(
    n_estimators=100,        # Số lượng cây, thử giảm nếu quá lớn
    max_depth=3,             # Giới hạn chiều sâu của cây
    learning_rate=0.05,       # Điều chỉnh tốc độ học
    min_samples_split=10,    # Điều chỉnh số mẫu tối thiểu cho mỗi split
    min_samples_leaf=5,      # Điều chỉnh số mẫu tối thiểu cho mỗi lá
    random_state=42
)

In [63]:
# Huấn luyện mô hình với dữ liệu huấn luyện
model.fit(X_train, y_train)

# Dự đoán trên tập huấn luyện và kiểm thử
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

# Đánh giá mô hình với RMSE và R²
train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))

train_r2 = r2_score(y_train, train_pred)
test_r2 = r2_score(y_test, test_pred)

print(f"Training RMSE: {train_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")
print(f"Training R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")

Training RMSE: 1.7269
Test RMSE: 2.0426
Training R²: 0.6839
Test R²: 0.5635


In [29]:
rmse_percentage = (test_rmse / 144453202.24) * 100
print(f"RMSE Percentage: {rmse_percentage:.2f}%")


RMSE Percentage: 0.00%


In [ ]:
# Cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Đánh giá mô hình bằng cross-validation cho RMSE
cross_val_score_model_rmse = cross_val_score(model, X_scaled, y, cv=kfold, scoring='neg_mean_squared_error')
cross_val_rmse = np.sqrt(-cross_val_score_model_rmse.mean())
print(f"Cross-validation RMSE: {cross_val_rmse:.4f}")

# Cross-validation (R²)
cross_val_score_model_r2 = cross_val_score(model, X_scaled, y, cv=kfold, scoring='r2')
cross_val_r2 = cross_val_score_model_r2.mean()
print(f"Cross-validation R²: {cross_val_r2:.4f}")

Cross-validation RMSE: 1.9730
Cross-validation R²: 0.5889
